In [2]:
import cdsapi
import xarray as xr
import pygrib #
import numpy as np
import pandas as pd
import time
import requests
import os
import xarray as xr
import zipfile


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly


import numpy as np
import matplotlib.pyplot as plt #
import matplotlib.colors #

from geopy.geocoders import Nominatim
from datetime import datetime
import timezonefinder
from astral.sun import sun
from astral.location import LocationInfo

## Download, prepare and plot the data

In this first section we download and prepare the data for the last 30 years, visualising it using different plots.

### Download and preparation

In [3]:


# select the years you want to download
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]


# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)


In [4]:
#Send API request and download data
c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'total_cloud_cover', 
                'total_precipitation',
                '2m_dewpoint_temperature',
                ],
    "area": [location.latitude + 1, 
             location.longitude - 1, 
             location.latitude - 1, 
             location.longitude + 1],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'past_climate.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved
    print(f"File saved at: {os.path.abspath(filename)}")

2023-12-15 11:41:39,193 INFO Welcome to the CDS
2023-12-15 11:41:39,194 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-12-15 11:41:39,325 INFO Request is completed


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/past_climate.grib


In [6]:
# List of variables to load
variables = ['2t','10v','10u','tp','tcc', '2d']

# Dictionary to hold the datasets
datasets = {}

# Open the GRIB file for each variable using the short name parameter
for var in variables:
    ds = xr.open_dataset('past_climate.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': var}})
    datasets[var] = ds

# Print the datasets just to check if everything worked
print(datasets)

{'2t': <xarray.Dataset>
Dimensions:     (time: 8928, latitude: 9, longitude: 9)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 38.96 38.71 38.46 38.21 ... 37.46 37.21 36.96
  * longitude   (longitude) float64 -3.435 -3.185 -2.935 ... -1.934 -1.684 358.6
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-12-15T12:11 GRIB to CDM+CF via cfgrib-0.9.1..., '10v': <xarray.Dataset>
Dimensions:     (time: 8928, latitude: 9, longitude: 9)
Co

### Rainfall and temperatures

In [58]:
#calculate RH from dewpoint and temperature
def rh(dewpoint, temperature):
    return 100*(np.exp((17.625*dewpoint)/(243.04+dewpoint))/np.exp((17.625*temperature)/(243.04+temperature)))

rh_all = rh(datasets['2d']['d2m']-273.15, datasets['2t']['t2m']-273.15)

datasets['rh'] = xr.Dataset({'rh': xr.DataArray(rh_all, coords=datasets['2d']['d2m'].coords, dims=datasets['2d']['d2m'].dims)})


In [19]:
# Calculate the climatology and average over latitude longitude and time  

avg_tp = datasets['tp']['tp'].groupby('time.month').mean(['time', 'latitude', 'longitude', 'step'])*30*1000
avg_temp = datasets['2t']['t2m'].groupby('time.month').mean(['time', 'latitude', 'longitude'])-273.15
max_temp = datasets['2t']['t2m'].groupby('time.month').max(['time', 'latitude', 'longitude'])-273.15
min_temp = datasets['2t']['t2m'].groupby('time.month').min(['time', 'latitude', 'longitude'])-273.15

avg_rh = datasets['rh']['rh'].groupby('time.month').min(['time', 'latitude', 'longitude'])
min_rh = datasets['rh']['rh'].groupby('time.month').mean(['time', 'latitude', 'longitude'])
max_rh = datasets['rh']['rh'].groupby('time.month').max(['time', 'latitude', 'longitude'])

avg_u = datasets['10u']['u10'].groupby('time.month').mean(['latitude', 'longitude'])
avg_v = datasets['10v']['v10'].groupby('time.month').mean(['latitude', 'longitude'])

In [21]:
# Create a subplot with shared x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add a bar chart for precipitation to the secondary y-axis
fig.add_trace(
    go.Bar(x=avg_tp.month, y=avg_tp, name='Precipitation', opacity=0.5),
    secondary_y=False,
)

# Add a line chart for temperature to the primary y-axis
fig.add_trace(
    go.Scatter(x=avg_temp.month, y=avg_temp, mode='lines', name='Temperature'),
    secondary_y=True, 
)

# Set the layout to have two y-axes
fig.update_layout(
    title = 'Average monthly temperature and precipitation',
    yaxis=dict(title='Precipitation (mm)'),
    yaxis2=dict(title='Temperature (°C)', overlaying='y', side='right'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

The blue bars show the total rainfall in mm averaged for each month.    
The orange line shows the average temperature in °C per month. 

In [19]:
#do do: revisit the title

print(plotly.__version__)

# Create a DataFrame from the DataArrays
df = pd.DataFrame({
    'month': avg_temp.month.values,
    'avg_temp': avg_temp.values,
    'max_temp': max_temp.values,
    'min_temp': min_temp.values
})

# Create a line chart for average temperature
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['month'], y=df['avg_temp'], 
                         mode='lines', 
                         name='Average temperature', 
                         line_color='orange')
                         )

# Add a line chart for max temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['max_temp'], 
                         mode='lines',
                         name='Average daily range per month', 
                         line_color='red')
                         )

# Add a line chart for min temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['min_temp'], 
                         mode='lines', 
                         name='Min temperature', 
                         line_color='red', 
                         fill='tonexty', 
                         fillcolor = 'rgba(255, 0, 0, 0.1)', 
                         showlegend=False)
                         )

# Add a line chart for min temperature
fig.add_hline(y=0, opacity=1, 
              line_width=2, 
              line_dash='dash', 
              line_color='blue',
              annotation_text='freezing', 
              annotation_position='top')

# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=df['month'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)
# Show the figure
fig.show()

5.18.0


This figure shows the average temperature for each month complemented by the average daily temperature range per month. Meaning: the top line shows the maximum temperature of every single day, averaged over the corresponding month. Same goes for the bottom line. In plain text: the CDS dataset provides us with an hourly mean temperature for each month. For this plot we took the highest, the lowest and the average values. The red area shows the temperature range over 24 h of an average day for each month.  
The line at 0 °C highlights freezing conditions.


### Wind speeds and directions

In [44]:
# Calculate wind speed
wind_speed = np.sqrt(avg_u**2 + avg_v**2)
#convert to km/h
wind_speed = wind_speed*3.6

# Calculate wind direction (see: https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398)
wind_direction = np.mod(180 + np.arctan2(avg_u, avg_v) * (180 / np.pi), 360)

#prepare the data for the wind rose
df = pd.DataFrame({'speed': wind_speed, 'direction': wind_direction})

bins_dir = np.linspace(0, 360, 9)
labels_dir = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
bins_speed = np.arange(0, df['speed'].max() + 1.1,  np.round(np.ceil(max(wind_speed.values))/5))
df['direction'] = pd.cut(df['direction'], bins=bins_dir, labels = labels_dir)
df['speed'] = pd.cut(df['speed'], bins=bins_speed)

# Calculate frequencies
frequency_df = df.groupby(['direction', 'speed']).size().reset_index(name='frequency')

# Calculate total frequency
total_frequency = frequency_df['frequency'].sum()

# Convert frequency to proportion
frequency_df['frequency'] = frequency_df['frequency'] / total_frequency

# Get the number of unique 'speed' categories
num_categories = len(frequency_df['speed'].unique())

# Sort the 'speed' categories
sorted_categories = frequency_df['speed'].sort_values().unique()


/var/folders/65/jjhht_kn6479d_8drvxjcgjh0000gn/T/ipykernel_34385/1095691032.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [45]:

# Create a custom color scale with the same number of colors as there are categories
custom_color_scale = plt.cm.viridis_r(np.linspace(0, 1, num_categories))

# Convert the color scale to a list of hex color strings
custom_color_scale = [matplotlib.colors.rgb2hex(color) for color in custom_color_scale]

# Define a color map for the sorted 'speed' categories
color_map = {category: color for category, color in zip(sorted_categories, custom_color_scale)}

# Create the wind rose chart
fig = px.bar_polar(frequency_df, 
                   r='frequency', 
                   theta='direction', 
                   color='speed', 
                   template='simple_white', 
                   color_discrete_map=color_map, labels={"speed": "Speed [km/h]"})  # Use the color map

# Update the layout to make it rectangular
fig.update_layout(
    width=800,  # Set the width to 700 pixels
    height=600,  # Set the height to 1000 pixels
    polar_radialaxis_showgrid=True,  # Show radial grid
    polar_angularaxis_showgrid=True  # Show angular grid
)

fig.show()

/Users/giacomo/.pyenv/versions/3.9.7/envs/climate_permaculture/lib/python3.9/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Each wind direction is represented by a bar. The length of the bar indicates how frequently the wind blows from that direction (in %).
The colours indicate the averaged wind speed in km/h. Keep in mind that these are averaged values and dont indicate how prone your location can be to events like storms.

## Average cloud cover and sunrise/sunset times

In [46]:
#Get rid of the latitude and longitude dimensions by averaging the data
avg_tcc = ds['tcc'].mean(dim=['longitude', 'latitude'])


#Now average the data of each hour of each month across the 30 years of data. We end up with 288 data points, representing 24 h per month
month_hour_grouped = avg_tcc.groupby(avg_tcc['time.month'] * 100 + avg_tcc['time.hour'])
hourly_mean_by_month = month_hour_grouped.mean(dim='time')


In [47]:
#find the timezone of the location
tf = timezonefinder.TimezoneFinder()
timezone_str = tf.certain_timezone_at(lat=location.latitude, lng=location.longitude)

#define location infos for the astral package only using coordinates
location_info = LocationInfo(None, None, timezone_str, location.latitude, location.longitude)

#define two empty lists for sunrise and sunset times
sunrise_times, sunset_times = [], []

# append sunrise and sunset times for the 15th of every month of 2022. Automatically adjusted for Daylight Saving Time (DST)
for month in range(1, 13):
    date = datetime(2022, month, 15)
    
    s = sun(location_info.observer, date=date, tzinfo=timezone_str)
    sunrise_times.append(s['sunrise'].strftime('%H:%M'))
    sunset_times.append(s['sunset'].strftime('%H:%M'))

print(sunrise_times, sunset_times)
    

['08:24', '08:00', '07:21', '07:34', '06:59', '06:46', '06:59', '07:25', '07:51', '08:18', '07:50', '08:18'] ['18:13', '18:48', '19:16', '20:45', '21:12', '21:33', '21:31', '21:02', '20:17', '19:32', '17:57', '17:51']


In [48]:
# Create a graph with cloud cover values plus sunrise and sunset times

# Get rid of the coding of the month/hour combination used for grouping before. 
month_values = hourly_mean_by_month['group'] // 100
hour_values = hourly_mean_by_month['group'] % 100

# Reshape the data to match the format expected by Plotly
data_reshaped = hourly_mean_by_month.values.reshape((12, 24)).T  # Use -1 to automatically infer the size

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=data_reshaped,
    x=list(range(12)),
    y=list(range(24)),
    xgap = 5,
    colorscale='gray_r',
    colorbar=dict(title="Cloud Cover"),
))

# Set x-axis tickvals and ticktext for each month
fig.update_xaxes(
    tickvals=list(range(len(month_values))),
    ticktext=[f"{month_name}" for month_name in ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]],
    tickmode='array',  # Use 'array' for custom tickvals and ticktext
    tickangle=-45,  # Rotate tick labels for better readability
)

# Set axis labels and title
fig.update_layout(
    title='Monthly hourly mean cloud cover with sunrise and sunset times',
    yaxis_title='Hour of the day',
    xaxis_title='Month',
)


# Set x-axis tickvals and ticktext for each day of the month


# Add a line for sunset times
fig.add_trace(go.Scatter(
    x=list(range(12)),
    y=[float('{:.2f}'.format(int(h) + int(m) / 60)) for h, m in [time.split(':') for time in sunset_times]],
    mode='lines',
    line=dict(color='rgb(150,0,255)', width=2),
    name='Sunset'
))

# Add a line for sunrise times
fig.add_trace(go.Scatter(
    x=list(range(12)),
    y=[float('{:.2f}'.format(int(h) + int(m) / 60)) for h, m in [time.split(':') for time in sunrise_times]],
    mode='lines',
    line=dict(color='rgb(255,65,0)', width=2),
    
    name='Sunrise'
))


# Update layout to show custom line in legend
fig.update_layout(
    showlegend=True,
    legend=dict(x=1.02, y=1.15),
    yaxis=dict(
        dtick=2,)
)


# Show the plot

fig.show()

This plot shows average hourly cloud cover for each month. Hourly changes of a month describe the cloud cover cycle of an average day for the associated month, while changes between months highlight the average seasonal trends.     \
The two lines show sunrise and sunset times, adjusted for the timezone of the location as well as daylight saving times.  

# PROJECTION DATA

In [78]:
start_year = 2016
end_year = 2046
year_range_forecast = [str(i) for i in range(start_year, end_year + 1)]


# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)

dataset_variables = ['air_temperature', 'relative_humidity', 'eastward_near_surface_wind', 'northward_near_surface_wind', 'precipitation']


In [79]:
# Send API requests for the data specified in dataset_variables. Each request returns a folder.zip

c = cdsapi.Client()

for variable in dataset_variables:
    #create an if statement, as certain variables also include level, while others dont
    if variable == 'air_temperature' or variable == 'relative_humidity':            
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                'level': '1000',
            },
            variable + '.zip')
        url = data.location

        # Download the file
        response = requests.get(url)

       # Check if the request was successful
        response.raise_for_status()
    else:         
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                
            },
            variable + '.zip')    
    # Get the location of the file to download
        url = data.location

        # Download the file
        response = requests.get(url)

        # Check if the request was successful
        response.raise_for_status()

    # If the request was successful, write the file
    filename = variable + '.zip'
    with open(filename, 'wb') as f:
        f.write(response.content)

        # Print the location where the file is saved
        print(f"File saved at: {os.path.abspath(filename)}")
print('Downloads completed')



2023-12-08 17:42:52,762 INFO Welcome to the CDS
2023-12-08 17:42:52,765 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-08 17:42:52,926 INFO Request is completed
2023-12-08 17:42:52,943 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data7/adaptor.esgf_wps.retrieve-1702041576.4923918-1567-9-345ed999-6a47-4245-8dd7-afa7f73852f8.zip to air_temperature.zip (221.1K)
2023-12-08 17:42:55,832 INFO Download rate 76.7K/s
2023-12-08 17:42:57,988 INFO Welcome to the CDS
2023-12-08 17:42:58,004 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\air_temperature.zip


2023-12-08 17:42:58,304 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data7/adaptor.esgf_wps.retrieve-1702049223.8629508-32692-19-fd778d2b-2a20-4e2f-b0d6-dd73d9691853.zip to relative_humidity.zip (219.3K)
2023-12-08 17:43:00,786 INFO Download rate 88.5K/s
2023-12-08 17:43:03,856 INFO Welcome to the CDS
2023-12-08 17:43:03,859 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\relative_humidity.zip


2023-12-08 17:43:04,267 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data7/adaptor.esgf_wps.retrieve-1701942513.2929933-10355-18-a23b5548-65b0-485d-b9f1-21103c8d344c.zip to eastward_near_surface_wind.zip (280.1K)
2023-12-08 17:43:06,967 INFO Download rate 103.8K/s
2023-12-08 17:43:09,439 INFO Welcome to the CDS
2023-12-08 17:43:09,455 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\eastward_near_surface_wind.zip


2023-12-08 17:43:09,650 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data0/adaptor.esgf_wps.retrieve-1701942543.392255-18791-11-4aed0407-db32-43fe-8dac-f512991603c5.zip to northward_near_surface_wind.zip (262.4K)
2023-12-08 17:43:12,138 INFO Download rate 105.6K/s
2023-12-08 17:43:14,347 INFO Welcome to the CDS
2023-12-08 17:43:14,350 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\northward_near_surface_wind.zip


2023-12-08 17:43:14,568 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.esgf_wps.retrieve-1701942563.4600832-24032-5-5809d856-40b9-4140-9496-7e423fb427fb.zip to precipitation.zip (263.1K)
2023-12-08 17:43:16,851 INFO Download rate 115.4K/s


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\precipitation.zip
Downloads completed


In [4]:
#extract all zip folders into new folders with the name of the variable they belong to

extract_dir = 'C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/'

for filename in os.listdir():

    if filename.endswith('.zip'):
        # Construct the full path to the zip file
        zip_file_path = os.path.join(os.getcwd(), filename)

        # Get the folder name from the zip file (excluding the '.zip' extension)
        folder_name = os.path.splitext(filename)[0]
        
        # Create a directory with the same name as the zip file within the parent directory
        extracted_dir = os.path.join(extract_dir, folder_name)

        os.makedirs(extracted_dir, exist_ok=True)

        # Open the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all the contents to the created directory
            zip_ref.extractall(extracted_dir)

        print(f"Files from {filename} extracted to: {extracted_dir}")

print("Extraction complete.")

Files from air_temperature.zip extracted to: C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/air_temperature
Files from eastward_near_surface_wind.zip extracted to: C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/eastward_near_surface_wind
Files from northward_near_surface_wind.zip extracted to: C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/northward_near_surface_wind
Files from precipitation.zip extracted to: C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/precipitation
Files from relative_humidity.zip extracted to: C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/relative_humidity
Extraction complete.


In [5]:
# find the paths to each .nc file, as they contain the data we need

nc_file_paths = []
for folder_name in os.listdir(extract_dir):
    folder_path = os.path.join(extract_dir, folder_name)

    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Find NC files within the folder
        nc_files = [file for file in os.listdir(folder_path) if file.endswith('.nc')]

        # Assuming there is exactly one NC file in each folder
        if len(nc_files) == 1:
            nc_file_paths.append(os.path.join(folder_path, nc_files[0]))

# Print the list of NC file paths
print("List of NC file paths:")
for nc_path in nc_file_paths:
    print(nc_path)

List of NC file paths:
C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/air_temperature\ta_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_20160116-20461216_v20210129.nc
C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/eastward_near_surface_wind\uas_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_20160116-20461216_v20210129.nc
C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/northward_near_surface_wind\vas_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_20160116-20461216_v20210129.nc
C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/precipitation\pr_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_20160116-20461216_v20210129.nc
C:/Users/matthias/Documents/Projects/permaculture-climate/prediction_data/relative_humidity\hur_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_20160116-20461216_v20210129.nc


In [32]:
#create a dictionary containing all datasets
projection_datasets = {}
for  path in nc_file_paths:
    #get the name of the folder the variable is saved in
    folder_path = os.path.dirname(path)
    folder_name = os.path.basename(folder_path)
    #open the nc file append it to projection_dataset dictionary
    proj_ds = xr.open_dataset(path)
    projection_datasets[folder_name] = proj_ds
    


In [37]:
print(projection_datasets['relative_humidity']['hur'].squeeze('plev').groupby('time.month').mean(['time', 'lat', 'lon']))

<xarray.DataArray 'hur' (month: 12)>
array([65.49983 , 60.91333 , 54.602455, 48.683865, 41.741577, 35.061726,
       29.595139, 33.333187, 47.585102, 56.292778, 65.03618 , 67.637146],
      dtype=float32)
Coordinates:
    plev     float64 1e+05
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    standard_name:  relative_humidity
    long_name:      Relative Humidity
    comment:        The relative humidity with respect to liquid water for T>...
    units:          %
    original_name:  RELHUM (using vinth2p_ecmwf)
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2021-01-28T23:59:33Z altered by CMOR: Converted type from...


In [38]:
#calculate average total precipitation
proj_avg_tp = projection_datasets['precipitation']['pr'].groupby('time.month').mean(['time', 'lat', 'lon']) *1000000

#calculate average, min and max avg temperature. Get rid of the pressure level dimesnion as it is not needed
proj_avg_temp = projection_datasets['air_temperature']['ta'].squeeze('plev').groupby('time.month').mean(['time', 'lat', 'lon'])-273.15
proj_max_temp = projection_datasets['air_temperature']['ta'].squeeze('plev').groupby('time.month').max(['time', 'lat', 'lon'])-273.15
proj_min_temp = projection_datasets['air_temperature']['ta'].squeeze('plev').groupby('time.month').min(['time', 'lat', 'lon'])-273.15

#Same operation but for relative humidity.
proj_avg_hum = projection_datasets['relative_humidity']['hur'].squeeze('plev').groupby('time.month').mean(['time', 'lat', 'lon'])
proj_max_hum = projection_datasets['relative_humidity']['hur'].squeeze('plev').groupby('time.month').max(['time', 'lat', 'lon'])
proj_min_hum = projection_datasets['relative_humidity']['hur'].squeeze('plev').groupby('time.month').min(['time', 'lat', 'lon'])

#calculate average wind speeds
proj_avg_u = projection_datasets['eastward_near_surface_wind']['uas'].groupby('time.month').mean(['time', 'lat', 'lon'])
proj_avg_v = projection_datasets['northward_near_surface_wind']['vas'].groupby('time.month').mean(['time', 'lat', 'lon'])



## Projected rainfall and temperature

In [76]:


# Create a subplot with shared x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add a bar chart for precipitation to the secondary y-axis
fig.add_trace(
    go.Bar(x=proj_avg_tp.month, y=proj_avg_tp, name='Precipitation', opacity=0.5),
    secondary_y=False,
)

# Add a line chart for temperature to the primary y-axis
fig.add_trace(
    go.Scatter(x=proj_avg_temp.month, y=proj_avg_temp, mode='lines', name='Temperature'),
    secondary_y=True,
)

# Set the layout to have two y-axes
fig.update_layout(
    title = 'Average projected temperature and precipitation',
    yaxis=dict(title='Precipitation (mm)'),
    yaxis2=dict(title='Temperature (°C)', overlaying='y', side='right'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=proj_avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

In [ ]:
# Create a DataFrame from the DataArrays
df = pd.DataFrame({
    'month': proj_avg_temp.month.values,
    'avg_temp': proj_avg_temp.values,
    'max_temp': proj_max_temp.values,
    'min_temp': proj_min_temp.values
})

# Create a line chart for average temperature
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['month'], y=df['avg_temp'], 
                         mode='lines', 
                         name='Average temperature', 
                         line_color='orange')
                         )

# Add a line chart for max temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['max_temp'], 
                         mode='lines',
                         name='Average daily range per month', 
                         line_color='red')
                         )

# Add a line chart for min temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['min_temp'], 
                         mode='lines', 
                         name='Min temperature', 
                         line_color='red', 
                         fill='tonexty', 
                         fillcolor = 'rgba(255, 0, 0, 0.1)', 
                         showlegend=False)
                         )

# Add a line chart for min temperature
fig.add_hline(y=0, opacity=1, 
              line_width=2, 
              line_dash='dash', 
              line_color='blue',
              annotation_text='freezing', 
              annotation_position='top')

# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=df['month'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)
# Show the figure
fig.show()

## Projected  relative humidity

In [74]:
# Create a DataFrame from the DataArrays
df = pd.DataFrame({
    'month': proj_avg_hum.month.values,
    'avg_hum': proj_avg_hum.values,
    'max_hum': proj_max_hum.values,
    'min_hum': proj_min_hum.values
})

# Create a line chart for average humidity
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['month'], y=df['avg_hum'], 
                         mode='lines', 
                         name='Average humidity', 
                         line_color='blue')
                         )

# Add a line chart for max humidity
fig.add_trace(go.Scatter(x=df['month'], y=df['max_hum'], 
                         mode='lines',
                         name='Average daily range per month', 
                         line_color='rgb(0,170,255)')
                         )

# Add a line chart for min humidity
fig.add_trace(go.Scatter(x=df['month'], y=df['min_hum'], 
                         mode='lines', 
                         name='Avg min humidity', 
                         line_color='rgb(0,170,255)', 
                         fill='tonexty', 
                         fillcolor = 'rgba(0, 0, 255, 0.1)', 
                         showlegend=False)
                         )



# Set the layout
fig.update_layout(
    yaxis=dict(title='Humidity (%)'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=df['month'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)
# Show the figure
fig.show()

## Projected wind speeds

In [109]:
# Calculate wind speed
wind_speed = np.sqrt(proj_avg_u**2 + proj_avg_v**2)
#convert to km/h
wind_speed = wind_speed*3.6

# Calculate wind direction (see: https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398)
wind_direction = np.mod(180 + np.arctan2(proj_avg_u, proj_avg_v) * (180 / np.pi), 360)

#prepare the data for the wind rose
df = pd.DataFrame({'speed': wind_speed, 'direction': wind_direction})

bins_dir = np.linspace(0, 360, 9)
labels_dir = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
bins_speed = np.arange(0, df['speed'].max() + 1.1,  np.round(np.ceil(max(wind_speed.values))/5))
df['direction'] = pd.cut(df['direction'], bins=bins_dir, labels = labels_dir)
df['speed'] = pd.cut(df['speed'], bins=bins_speed)

# Calculate frequencies
frequency_df = df.groupby(['direction', 'speed']).size().reset_index(name='frequency')

# Calculate total frequency
total_frequency = frequency_df['frequency'].sum()

# Convert frequency to proportion
frequency_df['frequency'] = frequency_df['frequency'] / total_frequency

# Get the number of unique 'speed' categories
num_categories = len(frequency_df['speed'].unique())

# Sort the 'speed' categories
sorted_categories = frequency_df['speed'].sort_values().unique()

C:\Users\matthias\AppData\Local\Temp\ipykernel_3908\4249272635.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [110]:
# Create a custom color scale with the same number of colors as there are categories
custom_color_scale = plt.cm.viridis_r(np.linspace(0, 1, num_categories))

# Convert the color scale to a list of hex color strings
custom_color_scale = [matplotlib.colors.rgb2hex(color) for color in custom_color_scale]

# Define a color map for the sorted 'speed' categories
color_map = {category: color for category, color in zip(sorted_categories, custom_color_scale)}

# Create the wind rose chart
fig = px.bar_polar(frequency_df, 
                   r='frequency', 
                   theta='direction', 
                   color='speed', 
                   template='simple_white', 
                   color_discrete_map=color_map, labels={"speed": "Speed [km/h]"})  # Use the color map

# Update the layout to make it rectangular
fig.update_layout(
    width=800,  # Set the width to 700 pixels
    height=600,  # Set the height to 1000 pixels
    polar_radialaxis_showgrid=True,  # Show radial grid
    polar_angularaxis_showgrid=True  # Show angular grid
)

fig.show()

c:\Users\matthias\miniconda3\envs\permaculture\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

